In [39]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [40]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
path = '/content/drive/MyDrive/zip1'

In [42]:
! pip install pygeohash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import numpy as np
import pandas as pd
from pandas import*
import datetime
import glob

# Data Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Data pre-processing libraries
from sklearn.preprocessing import LabelEncoder
import pygeohash as gh

In [ ]:
import os
dir_name = '/content/drive/MyDrive/zip'

extension = ".zip"

#folderpath = "/content/drive/MyDrive/zip/csv1/backup"
#os.mkdir(folderpath)
#for i, df in enumerate(my_list, 1):
#filename = "AIS_2021_01_2{}.csv".format(i)
    #filepath = os.path.join(folderpath, filename)
    #df.to_csv(filepath)
os.chdir('/content/drive/MyDrive/zip') # change directory from working dir to dir with files
for item in os.listdir(dir_name): 
  if item.endswith(extension): 
    file_name = os.path.abspath(item)
    os.remove(file_name)


In [ ]:
import os
import sys
import zipfile


def csv_files(source_dir):
    for filename in os.listdir(source_dir):
        if filename.endswith('.csv'):
            yield filename


source_dir = '/content/drive/MyDrive/dummy_csv'  # r'C:\users\xyz\Source\'
dest_dir = '/content/drive/MyDrive/zip'  # r'C:\users\xyz\Dest\'

os.chdir(source_dir)  # To work around zipfile limitations

for csv_filename in csv_files(source_dir):
    file_root = os.path.splitext(csv_filename)[0]
    zip_file_name = file_root + '.zip'
    zip_file_path = os.path.join(dest_dir, zip_file_name)
    with zipfile.ZipFile(zip_file_path, mode='w') as zf:
        zf.write(csv_filename)

In [45]:
import os, zipfile

dir_name = '/content/drive/MyDrive/zip1'

extension = ".zip"



os.chdir('/content/drive/MyDrive/zip1') # change directory from working dir to dir with files

for item in os.listdir(dir_name): 
  if item.endswith(extension): 
    file_name = os.path.abspath(item) # get full path of files
    zip_ref = zipfile.ZipFile(file_name) # create zipfile object
    zip_ref.extractall('/content/drive/MyDrive/zip1/csv') # extract file to dir

zip_ref.close() # close file

#os.remove(file_name) # de

In [46]:
path =  '/content/drive/MyDrive/zip1'

In [47]:
file_list = glob.glob(path + '/csv/*.*')
print(file_list)

['/content/drive/MyDrive/zip1/csv/AIS_2021_01_30.csv', '/content/drive/MyDrive/zip1/csv/AIS_2021_01_29.csv', '/content/drive/MyDrive/zip1/csv/AIS_2021_01_28.csv', '/content/drive/MyDrive/zip1/csv/AIS_2021_01_27.csv', '/content/drive/MyDrive/zip1/csv/AIS_2021_01_26.csv', '/content/drive/MyDrive/zip1/csv/AIS_2021_01_25.csv', '/content/drive/MyDrive/zip1/csv/AIS_2021_01_24.csv', '/content/drive/MyDrive/zip1/csv/AIS_2021_01_23.csv']


In [49]:
my_list = []
new_path = path + '/csv/*.csv'
for file in glob.glob(new_path):
  print(file)
  a = pd.read_csv(file)
  my_list.append(a)

/content/drive/MyDrive/zip1/csv/AIS_2021_01_30.csv
/content/drive/MyDrive/zip1/csv/AIS_2021_01_29.csv
/content/drive/MyDrive/zip1/csv/AIS_2021_01_28.csv
/content/drive/MyDrive/zip1/csv/AIS_2021_01_27.csv
/content/drive/MyDrive/zip1/csv/AIS_2021_01_26.csv
/content/drive/MyDrive/zip1/csv/AIS_2021_01_25.csv
/content/drive/MyDrive/zip1/csv/AIS_2021_01_23.csv


In [50]:
def drop_missing_values(data):
  data.isna().mean().round(4)*100
  data.dropna(inplace=True)
  data.isna().sum()
  data.drop_duplicates()
  #print(data)

In [51]:
def drop_unwanted_columns(data):
  data.drop(columns=['Heading',
       'VesselName', 'IMO', 'CallSign', 'Status', 'Length',
       'Width', 'Draft', 'Cargo', 'TransceiverClass'], axis = 1, inplace=True)

In [52]:
def drop_invalid_mmsi(data):
  
  invalid_entries=[111111111,222222222,333333333,444444444,555555555,666666666,777777777,888888888,999999999, 123456789, 987654321]
  invalid = data[data.MMSI.isin(invalid_entries)]
  data.drop(invalid.index, axis=0, inplace=True)
  #print(data)

In [53]:
def invalid_mmsi_length(data):
  data['MMSI_length']= data['MMSI'].map(str).apply(len)
  data = data[data['MMSI_length']==9]
  data.drop('MMSI_length', axis=1, inplace=True)

In [54]:
def zero_sog(data):
  index_names=data[data['SOG']==0.0].index
  data.drop(index_names, inplace=True)

In [55]:
def vesseltype_categories(data):
  data.dropna(subset = ["VesselType"], axis = 0, inplace=True)
  data['VesselType'] = data['VesselType'].apply(np.int64)
  data["VesselType"].replace({0: "Not Available", 30: "Fishing", 35:"Military", 
                          (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,23,24,25,26,27,28,29,33,34, 
                           38,39,40,41,42,43,44,45,46,47,48,49,50,51,53,54,55,56,57,58,59, 90,
                           91,92,93,94,95,96,97,98,99):'Other',
                          (60,61,62,63,64,65,66,67,68,69):'Passenger',
                          (70,71,72,73,74,75,76,77,78,79):'Cargo',
                          (36,37):'Pleasure Craft/Sailing',
                          (80,81,82,83,84,85,86,87,88,89):'Tanker',
                          (21,22,31,32,52):'Tug Tow'}, inplace=True)
  label_encoder = LabelEncoder()
  data['VesselType_labeled']=label_encoder.fit_transform(data['VesselType'])

In [56]:
def course_direction(data):
  def categorize(data):
    if data['COG']<22.5:
       return 'N'
    if data['COG']>=22.5 and data['COG']<67.5:
      return 'NE'
    if data['COG']>=67.5 and data['COG']<112.5:
      return 'E'
    if data['COG']>=112.5 and data['COG']<157.5:
      return 'SE'
    if data['COG']>=157.5 and data['COG']<202.5:
      return 'S'
    if data['COG']>=202.5 and data['COG']<247.5:
      return 'SW'
    if data['COG']>=247.5 and data['COG']<292.5:
      return 'W'
    if data['COG']>=292.5 and data['COG']<337.5:
      return 'NW'
    if data['COG']>=337.5:
      return 'N'
  data['COG_direction'] = data.apply(lambda data: categorize(data), axis=1)

In [57]:
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371000):
  if to_radians:
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
  a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
  return earth_radius * 2 * np.arcsin(np.sqrt(a))

In [58]:
def apply_haversine(data):
  data= data.sort_values(['MMSI', 'BaseDateTime'], ascending = True)
  data['distance'] = np.concatenate(data.groupby('MMSI').apply(lambda x: haversine(x['LAT'], x['LON'], x['LAT'].shift(), x['LON'].shift())).values)
  data['distance'] = data['distance'].fillna(0)
  data['dist_diff'] = data['distance'].diff()
  indexNames = data[ data['dist_diff'] <= 500 ].index # Get names of indexes for which column dist_diff has value less than or equal to 1
  data.drop(indexNames , inplace=True) # Delete these row indexes from dataFrame
  data['dist_diff'] = data['dist_diff'].fillna(0)

In [59]:
def geohash(data):
  data['geohash']=data.apply(lambda x: gh.encode(x.LAT, x.LON, precision=5), axis=1)

In [60]:
def Eda(data):
  drop_missing_values(data)
  drop_unwanted_columns(data)
  drop_invalid_mmsi(data)
  zero_sog(data)
  vesseltype_categories(data)
  geohash(data)
  course_direction(data)
  apply_haversine(data)
  print(data)


def readFiles():  
  length = len(my_list)
  for i in range(length):
    Eda(my_list[i])
readFiles()

#def readFiles():
 # length = len(my_list)
 # i = 0
  #while i < length:
     #Eda(my_list[i])    
  #i = i+1
#readFiles()

        MMSI         BaseDateTime       LAT       LON   SOG  COG VesselType  \
0  209729000  2021-09-30T00:04:24  27.90300 -82.43881  11.6  182      Cargo   
1  209729000  2021-09-30T00:03:20  27.90617 -82.43917   9.7  162      Cargo   
2  209729000  2021-09-30T00:05:30  27.89917 -82.43931  12.8  187      Cargo   
3  209729000  2021-09-30T00:06:41  27.89494 -82.43987  13.2  187      Cargo   
4  209729000  2021-09-30T00:07:51  27.89064 -82.44032  13.3  184      Cargo   
5  209729000  2021-09-30T00:09:00  27.88630 -82.44056  13.4  182      Cargo   
6  209729000  2021-09-30T00:10:10  27.88196 -82.44080  13.3  184      Cargo   

   VesselType_labeled geohash COG_direction  
0                   0   dhvqs             S  
1                   0   dhvqu             S  
2                   0   dhvqs             S  
3                   0   dhvqs             S  
4                   0   dhvqs             S  
5                   0   dhvqs             S  
6                   0   dhvqs             S  

In [61]:
import os
dir_name = '/content/drive/MyDrive/zip1/csv'

extension = ".csv"

folderpath = "/content/drive/MyDrive/zip1/csv/backup"
os.mkdir(folderpath)
for i, df in enumerate(my_list, 23):
    filename = "AIS_2022_01_{}.csv".format(i)
    filepath = os.path.join(folderpath, filename)
    df.to_csv(filepath)
    #os.chdir('/content/drive/MyDrive/zip/csv1') # change directory from working dir to dir with files

    #for item in os.listdir(dir_name): 
     # if item.endswith(extension): 
         # file_name = os.path.abspath(item)
   # os.remove(file_name)


In [62]:
new_file_list = glob.glob('/content/drive/MyDrive/zip1/csv/backup/*.csv')
print(new_file_list)

['/content/drive/MyDrive/zip1/csv/backup/AIS_2022_01_23.csv', '/content/drive/MyDrive/zip1/csv/backup/AIS_2022_01_24.csv', '/content/drive/MyDrive/zip1/csv/backup/AIS_2022_01_25.csv', '/content/drive/MyDrive/zip1/csv/backup/AIS_2022_01_26.csv', '/content/drive/MyDrive/zip1/csv/backup/AIS_2022_01_27.csv', '/content/drive/MyDrive/zip1/csv/backup/AIS_2022_01_28.csv', '/content/drive/MyDrive/zip1/csv/backup/AIS_2022_01_29.csv']


In [63]:
df_list = (pd.read_csv(file) for file in new_file_list)

In [65]:
big_df = pd.concat(map(pd.read_csv, glob.glob( '/content/drive/MyDrive/zip1/csv/backup/*.csv')))

In [66]:
print(big_df)

   Unnamed: 0       MMSI         BaseDateTime       LAT       LON   SOG  COG  \
0           0  209729000  2021-09-30T00:04:24  27.90300 -82.43881  11.6  182   
1           1  209729000  2021-09-30T00:03:20  27.90617 -82.43917   9.7  162   
2           2  209729000  2021-09-30T00:05:30  27.89917 -82.43931  12.8  187   
3           3  209729000  2021-09-30T00:06:41  27.89494 -82.43987  13.2  187   
4           4  209729000  2021-09-30T00:07:51  27.89064 -82.44032  13.3  184   
5           5  209729000  2021-09-30T00:09:00  27.88630 -82.44056  13.4  182   
6           6  209729000  2021-09-30T00:10:10  27.88196 -82.44080  13.3  184   
0           0  209729000  2021-09-30T00:11:21  27.87764 -82.44129  13.4  187   
1           1  209729000  2021-09-30T00:12:30  27.87336 -82.44188  13.4  187   
2           2  209729000  2021-09-30T00:13:41  27.86906 -82.44246  13.3  188   
3           3  209729000  2021-09-30T00:14:50  27.86482 -82.44311  13.3  188   
4           4  209729000  2021-09-30T00:

In [67]:
big_df.shape

(49, 11)

In [68]:
grouped_big_df = big_df.groupby(['geohash', 'VesselType', 'SOG']).agg({'VesselType':'count'})
grouped_big_df.columns = ['count']
grouped_big_df = grouped_big_df.reset_index()
grouped_big_df.head(100)

,geohash,VesselType,SOG,count
0,dhvmf,Cargo,12.9,2
1,dhvmf,Cargo,13.0,4
2,dhvq4,Cargo,13.5,8
3,dhvq4,Cargo,13.6,2
4,dhvq5,Cargo,13.0,1
5,dhvq5,Cargo,13.1,2
6,dhvq5,Cargo,13.2,2
7,dhvq5,Cargo,13.3,4
8,dhvq5,Cargo,13.4,1
9,dhvq5,Cargo,13.5,1


In [69]:
def my_agg(x):
  names = {'weighted_average_SOG': (x['count']*x['SOG']).sum()/x['count'].sum()}
  return pd.Series(names, index=['weighted_average_SOG'])

In [70]:
grouped_big_df.groupby(['geohash', 'VesselType']).apply(my_agg)

,,weighted_average_SOG
geohash,VesselType,
dhvmf,Cargo,12.966667
dhvq4,Cargo,13.520000
dhvq5,Cargo,13.245455
dhvq7,Cargo,13.100000
dhvqe,Cargo,13.333333
dhvqs,Cargo,13.000000
dhvqu,Cargo,9.700000
